In [2]:
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Function for recording data
def collect_points(N):
    motor_ls = []
    motor_ldots = []
    xy = []
    for i in range(N):
        input(f'Press enter to record point {i}')
        a, b, c = collect_data(0.5)
        motor_ls.append(a[-1])
        motor_ldots.append(b[-1])
        xy.append(c[-1])
        print(f'point {i}: ', motor_ls[-1], motor_ldots[-1], xy[-1])
    return np.array(motor_ls), np.array(motor_ldots), np.array(xy)


In [ ]:
# Collect 4 corners, in order: BR, TR, TL, BL
# For each of the 4 corners, put the brush in "painting" position and just barely touch the mullions with the brush.
USE_CORNERS = True
if False:
    # Collect some data
    ls4, ldots4, xy4 = collect_points(4)
    print(ls4)
    print(ldots4)
    print(xy4)
    # Save/Load data (in case of ipynb failure)
    np.savez('/tmp/4pts_data.npz', ls4=ls4, ldots4=ldots4, est4=xy4)
    np.savez(f'/tmp/4pts_data_{time.time()}.npz', ls4=ls4, ldots4=ldots4, est4=xy4)
else:
    with np.load('/tmp/4pts_data.npz') as f:
        ls4 = f['ls4']
        ldots4 = f['ldots4']
        xy4 = f['est4']

# print(ls4)
# print(ldots4)
# print(xy4)

In [4]:
PANE = 'CENTER_BOTTOM'

In [5]:
# 4 Corner Constants
# First define X1, X2, Y1, Y2 are the bounds of the pane relative to the bottom-left pulley.
if PANE == 'CENTER_BOTTOM':
    PULLEY_O = .037
    X1 = 1.887 - PULLEY_O
    X2 = 4.022 - PULLEY_O
    Y1 = 5.417 - 4.614
    Y2 = Y1 + 1.816
elif PANE == 'CENTER_TOP':
    raise NotImplementedError
elif PANE == 'LEFT_MID_HALF':
    raise NotImplementedError
elif PANE == 'RIGHT_MID_HALF':
    raise NotImplementedError

print(f'sanity check: pane width (84"=2.1336m): {X2 - X1 = }')
if 'CENTER' in PANE:
    print(f'              total width (5.833m): {X2 + X1 = }')

BRUSH_R = (3.1e-2 + 0.8e-2) / 2 # 3.1cm brush diameter, 0.8cm fuzz factor
baseTbrush = np.array([21.0e-2, 38.5e-2])       # brush in the base coordinate frame
baseTcdprrest = -np.array([0.5334, 0.381]) / 2  # cdpr has a carriagewidth/2 offset we need to correct for
XY4 = np.array([[X2 - BRUSH_R, Y1 + BRUSH_R],
                [X2 - BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y2 - BRUSH_R],
                [X1 + BRUSH_R, Y1 + BRUSH_R]]) - baseTbrush
print('Corner Locs: (using bottom-left coordinate system)')
print(XY4)

print('Expected Corner Locs: (using carriage-center coordinate system)')
print(XY4 - baseTcdprrest)
print('60mm buffer limits: (using carriage-center coordinate system)')
XY60 = np.array([[X2 - 0.06, Y1 + 0.06],
                 [X2 - 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y2 - 0.06],
                 [X1 + 0.06, Y1 + 0.06]]) - baseTbrush - baseTcdprrest
s = f'xLl{XY60[2][0]};xLr{XY60[0][0]};xLd{XY60[0][1]};xLu{XY60[1][1]}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')

sanity check: pane width (84"=2.1336m): X2 - X1 = 2.1350000000000002
              total width (5.833m): X2 + X1 = 5.835000000000001
Corner Locs: (using bottom-left coordinate system)
[[3.7555 0.4375]
 [3.7555 2.2145]
 [1.6595 2.2145]
 [1.6595 0.4375]]
Expected Corner Locs: (using carriage-center coordinate system)
[[4.0222 0.628 ]
 [4.0222 2.405 ]
 [1.9262 2.405 ]
 [1.9262 0.628 ]]
60mm buffer limits: (using carriage-center coordinate system)
xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996
gs0;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs1;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs2;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;gs0;xLl1.9667000000000001;xLr3.9817000000000005;xLd0.6685;xLu2.3644999999999996;


In [6]:
# Compute an xy homography correction using the 4 corners
def compute_H(act, est):
    # act is 2x4, est is 2x4
    A = np.zeros((8, 9))
    for i in range(4):
        A[2*i, 0:3] = act[:, i]
        A[2*i, 6:9] = -est[0, i] * act[:, i]
        A[2*i+1, 3:6] = act[:, i]
        A[2*i+1, 6:9] = -est[1, i] * act[:, i]
    _, _, V = np.linalg.svd(A)
    H = V[-1, :].reshape(3, 3)
    return H

def make_homogeneous(x):
    return np.vstack((x, np.ones(x.shape[1])))

In [7]:
H = compute_H(make_homogeneous(XY4.T), make_homogeneous(predicted_x.T))
print(H)

# Sanity check
def apply_H(good):
    pred = H @ make_homogeneous(good.T)
    pred /= pred[-1, :]
    return pred[:-1, :].T
print('applying homography to expected corners: ', apply_H(XY4))
print('actual measure corners:                  ', predicted_x)

NameError: name 'predicted_x' is not defined

In [ ]:
# Here are the limits we should use post-homography
print('60mm buffer limits:')
XY60_cdpr_coords = apply_H(XY60)
print(XY60_cdpr_coords)
xmin, xmax = min(XY60_cdpr_coords[:, 0]), max(XY60_cdpr_coords[:, 0])
ymin, ymax = min(XY60_cdpr_coords[:, 1]), max(XY60_cdpr_coords[:, 1])
s = f'xLl{xmin};xLr{xmax};xLd{ymin};xLu{ymax}'
print(s)
print(f'gs0;{s};gs1;{s};gs2;{s};gs0;{s};')
print('The following should be typed into gcode webpage:')
print('H:')
print(H.tolist())
print('Expected Mural Translation:')
print(np.min(XY60, axis=0))
print('Expected Mural Dimensions (with the given scale factor, this should be the total width & height):')
print(np.max(XY60, axis=0) - np.min(XY60, axis=0))
# print((np.max(XY60, axis=0) - np.min(XY60, axis=0)) + 0.12)

In [ ]:
plt.plot(*XY60.T, 'r*')
plt.plot(*XY60_cdpr_coords.T, 'k*')

In [ ]:
# To test the gcode homography, for the first
X, Y = 1355.851/1e3, 12.000/1e3
print('Original: ', X, Y)
X, Y = (X * 0.94379391 + 1.9667), (Y * 0.93413831 + 0.6685)
print(X, Y)
print('To be sent to the robot:')
apply_H(np.array([[X, Y]]).reshape(1, 2))